# Doc2Vec

## Import zone

In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle
import pandas as pd

C:\Users\sociocom\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Define parameter zone

In [2]:
path = r'C:\Users\sociocom\Desktop\Crawler\backup\31May-4June 2018'

In [3]:
path_play = r'C:\Users\sociocom\Desktop\Crawler\backup\playing'

In [4]:
grid_size = 10
diff = 0.1/grid_size

In [5]:
bkk_lat = 13.685780
bkk_long = 100.484605

In [6]:
kyt_lat = 34.931461
kyt_long = 135.685300

In [7]:
spl_lat = -23.600520
spl_long = -46.683309

In [8]:
sfc_lat = 37.738036
sfc_long = -122.485208

In [9]:
ham_lat = 53.501085
ham_long = 9.943682

In [10]:
#merge to 10 color
color_idx = {}
idx_color = {}

## Function zone

In [11]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [12]:
def clean_place(df, name):
    return df[df['place']!=name]

In [13]:
def separate_loc(df, flat, flong):
    df['ex_lat'] = df['ex_lat'].apply(lambda x: float(x))
    df['ex_long'] = df['ex_long'].apply(lambda x: float(x))
    data = []
    lat = flat
    long = flong
    for i in range(grid_size):
        for j in range(grid_size):
            data.append(df[(df['ex_lat'] >= lat) & (df['ex_lat'] < lat + diff) & (df['ex_long'] >= long) & (df['ex_long'] < long + diff)])
            idx = []
            idx.append((lat, long))
            idx.append((lat, long+diff))
            print(lat," ",long)
            lat += diff
            idx.append((lat,long))
            idx.append((lat, long+diff))
            u = i*10 + j
            idx.append(str(u))
            idx_color[str(u)] = idx
        long += diff
        lat = flat
    return data

In [14]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [15]:
def in_tuple(tp, x):
    return [z for y,z in tp if y == x]

## Data zone

In [159]:
df_spl = load_obj(path + r"\translated_spl_tweet")
df_bkk = load_obj(path + r"\translated_bkk_tweet")
df_ham = load_obj(path + r"\translated_ham_tweet")
df_sfc = load_obj(path + r"\translated_sfc_tweet")
df_kyt = load_obj(path + r"\translated_kyt_tweet")

In [160]:
df_bkk.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,137552452,ExpatDanBKK,1001726470962769920,"@David_Aamez Hmmm, I suspect Bill Heinecke wou...",2018-05-30 07:26:16,13.742844,100.5682602,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",1001725702192824321,710742220681293824,David_Aamez,en
1,372674276,NeungBellerin,1001726500725456897,Walk through (@Sam Market Market (Wat Phra Kae...,2018-05-30 07:26:23,13.724922,100.52291,Suriya Wong,"100.516253, 13.72236","100.536283, 13.730574",-1,-1,-1,th
2,1030302462,opalopongpang,1001726547160690689,I'm at Penguin Eat Shabu in Vadhana https://t....,2018-05-30 07:26:34,13.7388373,100.56067598,Vadhana,"100.55007, 13.732979","100.570733, 13.74842",-1,-1,-1,en
3,953277866661199872,Poppysuphannee,1001726564550168576,And it's not going to be a school ride. I do n...,2018-05-30 07:26:38,-1,-1,Frown,"100.469033, 13.773245","100.49389, 13.789877",-1,-1,-1,th
4,399872095,StoneSoupBKK,1001726646599041024,@TheMossadIL F@$king morons.,2018-05-30 07:26:58,-1,-1,Phra Khanong Nuea,"100.584345, 13.704628","100.606727, 13.729193",1001701922074701824,774174489458331649,TheMossadIL,en


## Data playing

In [217]:
df_bkk1 = load_obj(path_play + r"\translated_bkk_tweet1")
df_bkk2 = load_obj(path_play + r"\translated_bkk_tweet2")

In [218]:
df_bkk3 = pd.concat([df_bkk1, df_bkk2])

In [219]:
df_bkk3.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120,68120
unique,10746,10772,68120,59200,64013,2443,2426,149,104,102,16486,9040,9046,2
top,453950816,xiahjaashop,1004243074216677376,I do not know what to do.,2018-06-03 09:18:20,-1,-1,Thailand,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,442,442,1,861,5,62585,62585,4758,5172,5172,50536,49779,49779,56098


In [220]:
df_bkk3 = clean_place(df_bkk3, "Myanmar")
df_bkk3 = clean_place(df_bkk3, "Thailand")
df_bkk3 = clean_place(df_bkk3, "Bangkok")

In [221]:
df_bkk3.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286,58286
unique,8886,8902,58286,50273,55311,2311,2295,146,101,99,14574,7864,7869,2
top,453950816,xiahjaashop,1003057581764427776,I do not know what to do.,2018-06-08 06:27:52,-1,-1,Pathum Wan,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,442,442,1,791,5,53142,53142,3256,5172,5172,43171,42603,42603,49950


In [222]:
bkk2 = separate_loc(df_bkk3, bkk_lat, bkk_long)

13.68578   100.484605
13.69578   100.484605
13.705779999999999   100.484605
13.715779999999999   100.484605
13.725779999999999   100.484605
13.735779999999998   100.484605
13.745779999999998   100.484605
13.755779999999998   100.484605
13.765779999999998   100.484605
13.775779999999997   100.484605
13.68578   100.494605
13.69578   100.494605
13.705779999999999   100.494605
13.715779999999999   100.494605
13.725779999999999   100.494605
13.735779999999998   100.494605
13.745779999999998   100.494605
13.755779999999998   100.494605
13.765779999999998   100.494605
13.775779999999997   100.494605
13.68578   100.50460500000001
13.69578   100.50460500000001
13.705779999999999   100.50460500000001
13.715779999999999   100.50460500000001
13.725779999999999   100.50460500000001
13.735779999999998   100.50460500000001
13.745779999999998   100.50460500000001
13.755779999999998   100.50460500000001
13.765779999999998   100.50460500000001
13.775779999999997   100.50460500000001
13.68578   100.51460

In [223]:
len(idx_color)

100

In [224]:
idx_color["3"]

[(13.715779999999999, 100.484605),
 (13.715779999999999, 100.494605),
 (13.725779999999999, 100.484605),
 (13.725779999999999, 100.494605),
 '3']

### clean bkk

In [166]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,37095,37095,37095,37095,37095,37095,37095,37095,37095,37095,37095,37095,37095,37095
unique,7501,7514,37095,32641,34861,1399,1390,148,103,101,8699,5330,5332,2
top,940194643748143104,BaekhyunneeExol,1002950232445894656,I do not know what to do.,2018-06-03 09:18:20,-1,-1,Thailand,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,255,255,1,476,5,33985,33985,2653,3129,3129,27716,27313,27313,30186


In [167]:
df_bkk = df_bkk[df_bkk['place'] != 'Myanmar']
df_bkk = df_bkk[df_bkk['place']!='Thailand']
df_bkk = df_bkk[df_bkk['place']!='Bangkok']

In [168]:
df_bkk.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,31304,31304,31304,31304,31304,31304,31304,31304,31304,31304,31304,31304,31304,31304
unique,6206,6213,31304,27329,29725,1329,1321,145,100,98,7537,4590,4591,2
top,453950816,xiahjaashop,1003536454549565442,I do not know what to do.,2018-06-03 08:39:08,-1,-1,Pathum Wan,"100.528375, 13.72951","100.541027, 13.749708",-1,-1,-1,th
freq,248,248,1,437,4,28415,28415,1855,3129,3129,23499,23205,23205,26710


In [169]:
df_bkk["place"].value_counts()

Pathum Wan                 1855
Din Daeng                  1647
Pathumwan                  1356
Ratchathewi                1156
Phra Khanong               1083
Phaya Thai                 1031
Lumphini                   1029
Huai Khwang                 984
Phayathai                   819
Yannawa                     708
North Klongton              686
Vadhana                     642
Makkasan                    624
Silom                       552
Thanon Phaya Thai           535
Khlong Tan Nuea             522
Arun Amarin                 494
Khlong Toei                 482
Wattana                     478
Bangkok Noi                 470
Briefly                     426
Phayathai Road              412
Sathon                      386
Sathorn                     380
Bang Kapi                   376
Klong Toey                  370
Bang Pakok                  346
Thung Maha Mek              331
Si Phraya                   328
Phra Khanong Nuea           309
                           ... 
Bat Boar

In [170]:
"""bkk = []
for i in range(5):
    bkk.append(df_bkk[(df_bkk['ex_lat'] >= bkk_lat) & (df_bkk['ex_lat'] < bkk_lat + 0.02) & (df_bkk['ex_long'] >= bkk_long) & (df_bkk['ex_long'] < bkk_long + 0.02)])
    print(bkk_lat)
    bkk_lat += 0.02
    bkk_long += 0.02
"""

"bkk = []\nfor i in range(5):\n    bkk.append(df_bkk[(df_bkk['ex_lat'] >= bkk_lat) & (df_bkk['ex_lat'] < bkk_lat + 0.02) & (df_bkk['ex_long'] >= bkk_long) & (df_bkk['ex_long'] < bkk_long + 0.02)])\n    print(bkk_lat)\n    bkk_lat += 0.02\n    bkk_long += 0.02\n"

In [171]:
bkk = separate_loc(df_bkk, bkk_lat, bkk_long)

13.68578   100.484605
13.69578   100.484605
13.705779999999999   100.484605
13.715779999999999   100.484605
13.725779999999999   100.484605
13.735779999999998   100.484605
13.745779999999998   100.484605
13.755779999999998   100.484605
13.765779999999998   100.484605
13.775779999999997   100.484605
13.68578   100.494605
13.69578   100.494605
13.705779999999999   100.494605
13.715779999999999   100.494605
13.725779999999999   100.494605
13.735779999999998   100.494605
13.745779999999998   100.494605
13.755779999999998   100.494605
13.765779999999998   100.494605
13.775779999999997   100.494605
13.68578   100.50460500000001
13.69578   100.50460500000001
13.705779999999999   100.50460500000001
13.715779999999999   100.50460500000001
13.725779999999999   100.50460500000001
13.735779999999998   100.50460500000001
13.745779999999998   100.50460500000001
13.755779999999998   100.50460500000001
13.765779999999998   100.50460500000001
13.775779999999997   100.50460500000001
13.68578   100.51460

In [172]:
for x in bkk:
    print("#tweets", len(x))

#tweets 0
#tweets 8
#tweets 3
#tweets 18
#tweets 1
#tweets 10
#tweets 18
#tweets 20
#tweets 9
#tweets 7
#tweets 2
#tweets 4
#tweets 2
#tweets 14
#tweets 3
#tweets 8
#tweets 36
#tweets 82
#tweets 10
#tweets 8
#tweets 0
#tweets 4
#tweets 3
#tweets 9
#tweets 8
#tweets 46
#tweets 4
#tweets 2
#tweets 2
#tweets 3
#tweets 0
#tweets 2
#tweets 3
#tweets 20
#tweets 14
#tweets 23
#tweets 8
#tweets 4
#tweets 5
#tweets 6
#tweets 0
#tweets 1
#tweets 7
#tweets 41
#tweets 32
#tweets 361
#tweets 270
#tweets 26
#tweets 23
#tweets 2
#tweets 5
#tweets 11
#tweets 6
#tweets 8
#tweets 54
#tweets 98
#tweets 467
#tweets 91
#tweets 17
#tweets 15
#tweets 0
#tweets 1
#tweets 0
#tweets 8
#tweets 11
#tweets 54
#tweets 2
#tweets 0
#tweets 5
#tweets 10
#tweets 0
#tweets 0
#tweets 2
#tweets 32
#tweets 10
#tweets 139
#tweets 46
#tweets 2
#tweets 2
#tweets 12
#tweets 1
#tweets 1
#tweets 0
#tweets 10
#tweets 100
#tweets 21
#tweets 21
#tweets 42
#tweets 73
#tweets 4
#tweets 0
#tweets 0
#tweets 1
#tweets 16
#tweets 45
#twe

### clean kyoto

In [20]:
df_kyt.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,979626877244489728,B_CLASS_LV,1001817573917364224,@ rokonattuk Ments was dark.,2018-05-30 13:28:17,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",1001817447366840320,987313040281382912,rokonattuk,ja
1,2917764124,jnsg_0805,1001817576643624966,I was supposed to go to Kanjani live in summer...,2018-05-30 13:28:17,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",-1,-1,-1,ja
2,968012405757247488,jihyo__614,1001817631173894145,"@ 7NdOkP7T9URUGzD ー, Various",2018-05-30 13:28:30,-1,-1,Japan,"122.904034, 24.013343","153.997697, 45.562897",1001817154583318528,981075437450219520,7NdOkP7T9URUGzD,ja
3,879622055259328513,makotty1008,1001817644738244610,@ anikutomichoki ~ Please cheer for me Till th...,2018-05-30 13:28:34,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",1001813093331025921,972014359797891073,onikutomichoki,ja
4,874659432495435776,savanna3s_rx7,1001817680859553796,Kobayashi also does not change much ... Kobaya...,2018-05-30 13:28:42,-1,-1,Osaka,"135.091714, 34.271817","135.746799, 35.051352",-1,-1,-1,ja


In [21]:
df_kyt.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,34869,34869,34869,34869,34869,34869,34869,34869,34869,34869,34869,34869,34869,34869
unique,6599,6601,34869,34646,33048,2006,1993,293,270,270,12232,6704,6707,2
top,845996906568310784,koinchu_jp,1001998727551254528,Noisy,2018-05-31 13:27:36,-1,-1,Japan,"122.904034, 24.013343","153.997697, 45.562897",-1,-1,-1,ja
freq,350,350,1,48,4,29684,29684,10925,10953,10953,22385,21737,21737,34029


In [22]:
df_kyt['place'].unique()

array(['Kyoto City Nakagyo-ku', 'Japan', 'Osaka',
       'Kyoto City North District', 'Sakyo-ku', 'Sakyo',
       'Fushimi-ku, Kyoto-shi', 'Kinki area', 'Uji',
       'Kyoto City South District', 'Kamigyo-ku, Kyoto-shi', 'Osaka-fu',
       'Central place', 'Kamigyo-ku, Kyoto City', 'Kyoto-shi Minami-ku',
       'Kyoto-shi Kita-ku', 'Kyoto City Shimogyo Ward', 'Kami-ku',
       'Kyoto', 'Kyoto City Kamigyo Ward', 'Kyoto-shi Nakagyo', 'Kansai',
       'Kyoto City Fushimi Ward', 'Nagaokakyo', 'Kyoto City Area',
       'Nantan City', 'Tomaruka', 'Shiga',
       'Kyoto City, Dongshan District', 'Ukyo-ku', 'Kyoto-shi Nishikyo',
       'Chbu', 'JR Kyoto Station', 'Uji-shi',
       'The Imperial Household Agency Kyoto Office', 'Japn',
       'Kitano Tenmangu Shrine Tenjin', 'Kyoto-fu',
       'Ootama village (Adachi-gun)', '',
       'Eastern and western old-fashioned building of a tea ceremony classroom',
       'Flower Seeking Path', 'Shenglin Temple', 'Fushimi-Inari Taisha',
       'Fushimi

In [23]:
df_kyt2 = clean_place(df_kyt, "Japan")
df_kyt2 = clean_place(df_kyt2, "Kyoto")
df_kyt2 = clean_place(df_kyt2, "Osaka")

In [24]:
df_kyt2.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,22472,22472,22472,22472,22472,22472,22472,22472,22472,22472,22472,22472,22472,22472
unique,4403,4404,22472,22323,21663,1915,1902,290,269,269,7079,3880,3882,2
top,952880084712538112,pat_mimura,1001823558962561024,Noisy,2018-06-01 09:00:05,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",-1,-1,-1,ja
freq,346,346,1,48,4,17533,17533,2743,2966,2966,15276,15119,15119,21883


In [25]:
df_kyt2['place'].unique()

array(['Kyoto City Nakagyo-ku', 'Kyoto City North District', 'Sakyo-ku',
       'Sakyo', 'Fushimi-ku, Kyoto-shi', 'Kinki area', 'Uji',
       'Kyoto City South District', 'Kamigyo-ku, Kyoto-shi', 'Osaka-fu',
       'Central place', 'Kamigyo-ku, Kyoto City', 'Kyoto-shi Minami-ku',
       'Kyoto-shi Kita-ku', 'Kyoto City Shimogyo Ward', 'Kami-ku',
       'Kyoto City Kamigyo Ward', 'Kyoto-shi Nakagyo', 'Kansai',
       'Kyoto City Fushimi Ward', 'Nagaokakyo', 'Kyoto City Area',
       'Nantan City', 'Tomaruka', 'Shiga',
       'Kyoto City, Dongshan District', 'Ukyo-ku', 'Kyoto-shi Nishikyo',
       'Chbu', 'JR Kyoto Station', 'Uji-shi',
       'The Imperial Household Agency Kyoto Office', 'Japn',
       'Kitano Tenmangu Shrine Tenjin', 'Kyoto-fu',
       'Ootama village (Adachi-gun)', '',
       'Eastern and western old-fashioned building of a tea ceremony classroom',
       'Flower Seeking Path', 'Shenglin Temple', 'Fushimi-Inari Taisha',
       'Fushimi Kandakara', 'Fushimi-Momyama Cast

In [26]:
df_kyt2.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,979626877244489728,B_CLASS_LV,1001817573917364224,@ rokonattuk Ments was dark.,2018-05-30 13:28:17,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",1001817447366840320,987313040281382912,rokonattuk,ja
1,2917764124,jnsg_0805,1001817576643624966,I was supposed to go to Kanjani live in summer...,2018-05-30 13:28:17,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",-1,-1,-1,ja
3,879622055259328513,makotty1008,1001817644738244610,@ anikutomichoki ~ Please cheer for me Till th...,2018-05-30 13:28:34,-1,-1,Kyoto City Nakagyo-ku,"135.730513, 34.99342","135.779805, 35.018685",1001813093331025921,972014359797891073,onikutomichoki,ja
5,2272992612,kaketemiho9696,1001817781850005504,I am rude to Professor @ konomikimura but as a...,2018-05-30 13:29:06,-1,-1,Kyoto City North District,"135.655229, 35.018131","135.779387, 35.17085",1001796881805004800,219296398,konomikimura,ja
6,710314807140835329,Ah4Ut,1001817790825680896,Bamboo returned.,2018-05-30 13:29:08,-1,-1,Sakyo-ku,"135.725501, 35.00634","135.860702, 35.32117",-1,-1,-1,ja


In [27]:
kyt = separate_loc(df_kyt2, kyt_lat, kyt_long)

34.931461   135.6853
34.951461   135.6853
34.971461000000005   135.6853
34.99146100000001   135.6853
35.01146100000001   135.6853
34.931461   135.70530000000002
34.951461   135.70530000000002
34.971461000000005   135.70530000000002
34.99146100000001   135.70530000000002
35.01146100000001   135.70530000000002
34.931461   135.72530000000003
34.951461   135.72530000000003
34.971461000000005   135.72530000000003
34.99146100000001   135.72530000000003
35.01146100000001   135.72530000000003
34.931461   135.74530000000004
34.951461   135.74530000000004
34.971461000000005   135.74530000000004
34.99146100000001   135.74530000000004
35.01146100000001   135.74530000000004
34.931461   135.76530000000005
34.951461   135.76530000000005
34.971461000000005   135.76530000000005
34.99146100000001   135.76530000000005
35.01146100000001   135.76530000000005


In [28]:
for x in kyt:
    print("#tweets", len(x))

#tweets 9
#tweets 28
#tweets 30
#tweets 4
#tweets 12
#tweets 27
#tweets 23
#tweets 8
#tweets 38
#tweets 15
#tweets 13
#tweets 15
#tweets 60
#tweets 213
#tweets 307
#tweets 61
#tweets 110
#tweets 771
#tweets 230
#tweets 132
#tweets 69
#tweets 83
#tweets 45
#tweets 425
#tweets 367


### clean spl

In [29]:
df_spl.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,880139242558676993,gu3ds_,1002070123061051392,I just think it's funny how the debauch hj ta ...,2018-05-31 06:11:49,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
1,71562250,Alexrodrigues73,1002070143873245184,@IvanCarlus @eaglesaopaulo kkkkkkkkk.... cai m...,2018-05-31 06:11:54,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",1002023363378851841,47545111,IvanCarlus,pt
2,883893293465186304,ester_querinoo,1002070146721222657,"Mother, everything that comes from you to me i...",2018-05-31 06:11:55,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
3,4794804172,elizaberh_costa,1002070147404828672,General sleeping,2018-05-31 06:11:55,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
4,33572317,thelukeset,1002070200508932096,people this series 13 reasons why eh mto compl...,2018-05-31 06:12:08,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt


In [30]:
df_spl.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,121964,121964,121964,121964,121964,121964,121964,121964,121964,121964,121964,121964,121964,121964
unique,11835,11873,121964,120843,98040,1767,1760,256,260,260,35890,18627,18647,2
top,139218031,ViniMelchior,1003306730938388485,Good Morning,2018-06-03 15:27:12,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
freq,528,528,1,82,9,118668,118668,113608,112844,112844,82906,81153,81153,115129


In [31]:
df_spl['place'].unique()

array(['Sao Paulo', 'Sao Caetano do Sul', 'Villa Mariana', 'Brazil',
       'Privacy Policy', 'Space of Americas - Touching Front', 'Home',
       'Anhangabaú Valley', 'Itaim Bibi', 'Pine trees',
       'Institute of Infectology Emílio Ribas',
       'Heroes Square of the Brazilian Expeditionary Force', 'Bom Retiro',
       'Real Estate Portal', 'Bella Paulista', 'Golden Tulip Belas Artes',
       'Perdizes', 'Hotel Metropolis', 'Santa Cecilia',
       'Mount Lebanon Bakery', 'Football Museum', 'Muji', 'About Me',
       'Upgrade Club', 'Ibirapuera park', 'Nice view', 'Lions Nightclub',
       'St. Paul', 'Made', 'Recanto dos Brunos - Municipal Market',
       'Cabana Burger', 'Mamoru sushi', 'Santos Yacht Club',
       'Consolation', 'Teatro Arena Eugenio KusNet', 'Hotel Hilton',
       'Augusta Street', 'About Us |', 'Moema',
       'Museum of Contemporary Art (MAC-USP)', 'Smart Fit',
       'About Us | (I.e.', 'Hotel Lido Plaza', 'America',
       'IBM Customer Center - ICC', 'Renai

In [32]:
df_spl = clean_place(df_spl, "Sao Paulo")

In [33]:
df_spl.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,8356,8356,8356,8356,8356,8356,8356,8356,8356,8356,8356,8356,8356,8356
unique,1326,1328,8356,8337,8226,40,40,255,259,259,3014,1959,1959,2
top,763471837611302912,perrykingdom,1002628037643554824,I miss you,2018-06-03 16:03:38,-1,-1,Brazil,"-73.991482, -33.751051","-32.378186, 5.27192",-1,-1,-1,pt
freq,215,215,1,3,3,8311,8311,5902,5902,5902,5000,4758,4758,7457


In [34]:
df_spl = clean_place(df_spl, "Brazil")

In [35]:
df_spl.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,2454,2454,2454,2454,2454,2454,2454,2454,2454,2454,2454,2454,2454,2454
unique,656,656,2454,2451,2436,30,30,254,258,258,696,519,519,2
top,980117675480436736,MarceloPaolill2,1002505063833788416,@ 1D_Adults follows me back,2018-06-02 22:45:13,-1,-1,Sao Caetano do Sul,"-46.584984, -23.650508","-46.544757, -23.599084",-1,-1,-1,pt
freq,174,174,1,2,3,2420,2420,1593,1593,1593,1717,1678,1678,2322


In [36]:
spl = separate_loc(df_spl, spl_lat, spl_long)

-23.60052   -46.683309
-23.58052   -46.683309
-23.56052   -46.683309
-23.54052   -46.683309
-23.52052   -46.683309
-23.60052   -46.663309
-23.58052   -46.663309
-23.56052   -46.663309
-23.54052   -46.663309
-23.52052   -46.663309
-23.60052   -46.643308999999995
-23.58052   -46.643308999999995
-23.56052   -46.643308999999995
-23.54052   -46.643308999999995
-23.52052   -46.643308999999995
-23.60052   -46.62330899999999
-23.58052   -46.62330899999999
-23.56052   -46.62330899999999
-23.54052   -46.62330899999999
-23.52052   -46.62330899999999
-23.60052   -46.60330899999999
-23.58052   -46.60330899999999
-23.56052   -46.60330899999999
-23.54052   -46.60330899999999
-23.52052   -46.60330899999999


In [37]:
for x in spl:
    print("#tweets", len(x))

#tweets 0
#tweets 1
#tweets 0
#tweets 1
#tweets 0
#tweets 1
#tweets 1
#tweets 1
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 2
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0
#tweets 0


### clean sfc

In [38]:
df_sfc.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,219890004,fantompower,1002026082013347840,@vashikoo @flashmurphy check it out,2018-05-31 03:16:49,-1,-1,San Francisco,"-122.514926, 37.708075","-122.357031, 37.833238",1002025718811803648,35083292,vashikoo,en
1,2574858916,dafeuvieghara,1002026155979882497,Damn I really wanted Tyler up up that choppa o...,2018-05-31 03:17:06,-1,-1,California,"-124.482003, 32.528832","-114.131212, 42.009519",-1,-1,-1,en
2,24658412,nguyenstephane,1002026166977380353,@KurtWagner8 She did that to more than one!,2018-05-31 03:17:09,-1,-1,San Francisco,"-122.514926, 37.708075","-122.357031, 37.833238",1001988536843886599,359425864,KurtWagner8,en
3,1589692776,SF311Reports,1002026166805581824,Opened Homeless Concerns request via iphone at...,2018-05-31 03:17:09,37.77284359,-122.44161274,San Francisco,"-122.514926, 37.708075","-122.357031, 37.833238",-1,-1,-1,en
4,775257167129554944,WestCoastIzzy_,1002026176167067648,“It means get lost squidward” lmao favorite li...,2018-05-31 03:17:11,-1,-1,California,"-124.482003, 32.528832","-114.131212, 42.009519",-1,-1,-1,en


In [39]:
df_sfc.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,68534,68534,68534,68534,68534,68534,68534,68534,68534,68534,68534,68534,68534,68534
unique,15655,15665,68534,68222,60304,5727,5679,507,512,512,22548,16169,16174,1
top,1589692776,SF311Reports,1002250905637416961,I've got the #Warriors. Who ya got?,2018-06-01 03:33:23,-1,-1,California,"-124.482003, 32.528832","-114.131212, 42.009519",-1,-1,-1,en
freq,5336,5336,1,50,5,59385,59385,33461,33472,33472,44398,42702,42702,68534


In [40]:
df_sfc['place'].unique()

array(['San Francisco', 'California', 'SoMa StrEat Food Park',
       'Ashbury Heights', 'Yerba Buena Gardens', 'Civic Center',
       'Apartment SCCS', "Charmaine's", 'Twitter HQ',
       'Mission Dolores Park', 'Pearl', 'Lower Haight',
       'Golden Gate Bridge', 'Downtown', 'The Masonic', 'Tenderloin',
       'Miraloma', 'Laurel Heights', 'Golden Gate', 'Forest Hill',
       'Hilton San Francisco Union Square Pool', 'SoMa',
       'Presidio Heights', 'Victoria Theatre', 'South of Market',
       'Fitness SF MidMarket', 'The Fillmore', 'Duboce Triangle',
       'South Beach', 'Western Addition', 'The Den', 'Alamo Square',
       'The Haight', 'The Independent', 'Noe Valley',
       'Great American Music Hall', 'Castro Theatre', 'Mission Bay',
       'Wing Wings', 'Upper Market', 'Lower Pacific Heights',
       'Baker Beach', 'Social Hall SF', 'Lake Street', 'Dogpatch',
       '10 Golden Gate Transit', 'Museum Of Ice Cream', 'Pacific Heights',
       'Union Square', 'UCSF', 'San Fran

In [41]:
df_sfc = clean_place(df_sfc, "California")
df_sfc = clean_place(df_sfc, "San Francisco")

In [42]:
df_sfc.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,2541,2541,2541,2541,2541,2541,2541,2541,2541,2541,2541,2541,2541,2541
unique,935,935,2541,2538,2516,20,20,505,511,511,733,627,627,1
top,758415480881164288,test5geo1798,1003104585991348224,@twitterpiano #request PIANO MAN BY BILLY JOEL,2018-06-03 17:50:02,-1,-1,Twitter HQ,"-122.416799, 37.776888","-122.416799, 37.776888",-1,-1,-1,en
freq,399,399,1,2,3,2509,2509,460,460,460,1792,1710,1710,2541


In [43]:
sfc = separate_loc(df_sfc, sfc_lat, sfc_long)

37.738036   -122.485208
37.758036000000004   -122.485208
37.77803600000001   -122.485208
37.79803600000001   -122.485208
37.81803600000001   -122.485208
37.738036   -122.465208
37.758036000000004   -122.465208
37.77803600000001   -122.465208
37.79803600000001   -122.465208
37.81803600000001   -122.465208
37.738036   -122.44520800000001
37.758036000000004   -122.44520800000001
37.77803600000001   -122.44520800000001
37.79803600000001   -122.44520800000001
37.81803600000001   -122.44520800000001
37.738036   -122.42520800000001
37.758036000000004   -122.42520800000001
37.77803600000001   -122.42520800000001
37.79803600000001   -122.42520800000001
37.81803600000001   -122.42520800000001
37.738036   -122.40520800000002
37.758036000000004   -122.40520800000002
37.77803600000001   -122.40520800000002
37.79803600000001   -122.40520800000002
37.81803600000001   -122.40520800000002


In [44]:
for x in sfc:
    print(len(x))

1
2
0
1
0
0
0
0
0
0
8
0
0
0
0
0
14
1
0
0
0
1
2
1
0


### clean ham

In [45]:
df_ham.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,855881532145360896,focus_regional,1002062196292182016,Police Düren - driver flung out of the car aft...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
1,855881532145360896,focus_regional,1002062196334096384,Police Neustadt bei Coburg - Without license o...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
2,855881532145360896,focus_regional,1002062196317270017,Police Gädheim - gelding with enigmatic neck i...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
3,855881532145360896,focus_regional,1002062196304695297,Polizeipräsidium Südosthessen - Offenbach - Pr...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
4,855881532145360896,focus_regional,1002062196296355840,Police Neustadt bei Coburg - Accident Exodus h...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de


In [46]:
df_ham.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,6835,6835,6835,6835,6835,6835,6835,6835,6835,6835,6835,6835,6835,6835
unique,1019,1019,6835,6821,6266,179,182,80,81,81,3063,1928,1928,2
top,855881532145360896,focus_regional,1002556642490281984,Want to work at Essity? We're #hiring in #Hamb...,2018-06-01 02:45:24,-1,-1,Hamburg,"8.42016, 53.395118","10.325199, 53.964655",-1,-1,-1,de
freq,644,644,1,3,25,6443,6443,5008,5000,5000,3686,3570,3570,4666


In [47]:
df_ham['place'].unique()

array(['Germany', 'Hamburg', 'Lower Saxony', 'Neustadt', 'Sankt Pauli',
       'Eimsbüttel', 'Frankfurt School Of Finance & Management',
       'Hotel Atlantic Kempinski', 'Klostertor', 'Harvestehude',
       'Alsterperle', 'Saint George', 'Bucerius Law School',
       'Bistro La Forchetta', 'Winterhude', 'Hamburg central station',
       'North Sea', 'Carlson', 'Rotherbaum', 'Altstadt',
       'elbphilharmonie', 'Upper Hafenquartier', 'chamber of Commerce',
       'Alex', 'Barmbek-Nord', 'Rolf-Liebermann-Studio',
       'Nord Event Panoramadeck', 'HafenCity', 'Schleswig-Holstein',
       'Eppendorf', 'Hoheluft-West', 'Underground station Emilienstraße',
       'Stadtpark', 'Reeperbahn', 'Moorburg', 'Körber Foundation',
       'Block House', 'Empire Riverside Hotel',
       'Finnische Seemannskirche - Hamburg Maritime Church',
       'Landungsbrücken', 'Theater Kehrwieder', 'Mrs. Moller',
       'Alsterdorf', 'Great freedom 36', 'Stage Operettenhaus',
       "St. Michael's Church", 'Mi

In [48]:
df_ham = clean_place(df_ham, "Hamburg")

In [49]:
df_ham["place"].value_counts()

Germany                                  1464
Schleswig-Holstein                         46
Barmbek-Nord                               37
HafenCity                                  30
Eimsbüttel                                 25
Rotherbaum                                 24
Eppendorf                                  23
Klostertor                                 20
Saint George                               14
Harvestehude                               13
Alsterdorf                                 12
Hamburg central station                     9
Lower Saxony                                9
Sankt Pauli                                 7
Neustadt                                    5
Upper Hafenquartier                         4
Altstadt                                    4
elbphilharmonie                             4
Stage Operettenhaus                         3
Eilbek                                      3
Bucerius Law School                         3
Altonaer fish market              

In [50]:
df_ham = clean_place(df_ham, "Germany")
df_ham = clean_place(df_ham, "Schleswig-Holstein")

In [51]:
df_ham.describe()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
count,317,317,317,317,317,317,317,317,317,317,317,317,317,317
unique,93,93,317,317,317,2,2,77,77,77,146,116,116,2
top,11988402,klapauzius,1002447627563257856,Yesterday Ikaruga appeared for the Switch. It ...,2018-06-02 15:30:12,-1,-1,Barmbek-Nord,"10.032526, 53.582949","10.069975, 53.607996",-1,-1,-1,de
freq,31,31,1,1,1,316,316,37,37,37,171,168,168,247


In [52]:
ham = separate_loc(df_ham, ham_lat, ham_long)

53.501085   9.943682
53.52108500000001   9.943682
53.54108500000001   9.943682
53.56108500000001   9.943682
53.581085000000016   9.943682
53.501085   9.963682
53.52108500000001   9.963682
53.54108500000001   9.963682
53.56108500000001   9.963682
53.581085000000016   9.963682
53.501085   9.983682
53.52108500000001   9.983682
53.54108500000001   9.983682
53.56108500000001   9.983682
53.581085000000016   9.983682
53.501085   10.003682
53.52108500000001   10.003682
53.54108500000001   10.003682
53.56108500000001   10.003682
53.581085000000016   10.003682
53.501085   10.023681999999999
53.52108500000001   10.023681999999999
53.54108500000001   10.023681999999999
53.56108500000001   10.023681999999999
53.581085000000016   10.023681999999999


In [53]:
for x in ham:
    print(len(x))

0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## Prepare train-test data

In [225]:
count = 0
for x in bkk2:
    count += len(x)
    print("#tweets in area : " ,len(x))

#tweets in area :  0
#tweets in area :  9
#tweets in area :  6
#tweets in area :  27
#tweets in area :  3
#tweets in area :  18
#tweets in area :  100
#tweets in area :  30
#tweets in area :  24
#tweets in area :  14
#tweets in area :  2
#tweets in area :  7
#tweets in area :  5
#tweets in area :  32
#tweets in area :  11
#tweets in area :  15
#tweets in area :  81
#tweets in area :  146
#tweets in area :  23
#tweets in area :  21
#tweets in area :  0
#tweets in area :  8
#tweets in area :  3
#tweets in area :  24
#tweets in area :  15
#tweets in area :  75
#tweets in area :  8
#tweets in area :  6
#tweets in area :  9
#tweets in area :  6
#tweets in area :  1
#tweets in area :  5
#tweets in area :  5
#tweets in area :  41
#tweets in area :  28
#tweets in area :  42
#tweets in area :  11
#tweets in area :  5
#tweets in area :  6
#tweets in area :  11
#tweets in area :  1
#tweets in area :  1
#tweets in area :  8
#tweets in area :  91
#tweets in area :  81
#tweets in area :  592
#tweets

In [226]:
print(count)

4683


In [227]:
bkk2[1]['text']

12556    I'm at Anantara Shuttle Boat in Thonburi, Bang...
16948               I'm at SKYLINE https://t.co/Ab6620SgTY
17911    I'm at AvaniFit in Bangkok https://t.co/jRiIY2...
19804    I'm at Bukkhalo Intersection in Thon Buri, Ban...
21578    I'm at Riverside Plaza in Thon Buri, Bangkok h...
21663    I'm at ThaiExpress in Thon Buri, Bangkok https...
21826              I'm at Attitude https://t.co/QNVi9y0mqF
30844    I'm at Riverside Plaza in Thon Buri, Bangkok h...
27534    Just posted a photo @ ... https://t.co/DhA9hlE02X
Name: text, dtype: object

In [228]:
"Bangkok https://t.co/CzaG6Fes2S."[:7]

'Bangkok'

In [229]:
bkk_str = []
for y in bkk2:
    doc = ""
    for x in y['text']:
        if "https://" in x.lower():
            idx = (x.lower().find("https://"))-1
            x = x[:idx]
            doc += x + ". "
    doc += " \n"
    bkk_str.append(doc)

In [230]:
bkk_str[23]

'Cant start the meeting without selfie with all members of the Indonesian delegation to the 20th Technical WG on Tra…. "Tari Kipas" welcoming Ministers and Senior Officials participating the 20th Meeting on Transboundary Haze Pollutio…. I\'m at Assumption Convent School in Bang Rak, Bangkok. I\'m at Saphan Taksin Park in Bang Rak, Bangkok. Hot Hot @ Sathorn (Taksin) in Bang Rak, Bangkok @. I\'m at M.5/2. I\'m at Assumption Convent School in Bang Rak, Bangkok.  (@ The Oriental Thai Cooking School in Khlong San, Bangkok). @ BTS Saphan Taksin - @bts_skytrain in Bang Rak, Bangkok w / @por_iiii). Come again (@Jung Ju noodle in Khlong San, Bangkok). I\'m at Assumption Convent School in Bang Rak, Bangkok. I\'m at South Prakan. I will have dinner (@ noodle noodles in Khlong San, Bangkok). I\'m at Assumption Convent School in Bang Rak, Bangkok. I like to eat a lot (@ fried pork is very delicious.. I\'m at The Oriental Thai Cooking School in Khlong San, Bangkok. I\'m at Royal Park in Bangkok, Ba

In [231]:
bkk_str[0]

' \n'

In [232]:
f = open(path_play + "/bkk_demo_train","w",encoding="utf-8")
#file = open(path + "bkk_demo_test","w",encoding="utf-8")
for x in bkk_str:
    f.write(x)
    print(x)
f.close()

 

I'm at Anantara Shuttle Boat in Thonburi, Bangkok. I'm at SKYLINE. I'm at AvaniFit in Bangkok. I'm at Bukkhalo Intersection in Thon Buri, Bangkok. I'm at Riverside Plaza in Thon Buri, Bangkok. I'm at ThaiExpress in Thon Buri, Bangkok. I'm at Attitude. I'm at Riverside Plaza in Thon Buri, Bangkok. Just posted a photo @ ....  

I'm at Somdejprapinklao Hospital in Thon Buri, Bangkok. I'm at Somdejprapinklao Hospital in Thon Buri, Bangkok. The first dental implant (@ dentist dentist). I'm at Somdejprapinklao Hospital in Thon Buri, Bangkok. I'm at Somdejprapinklao Hospital in Thon Buri, Bangkok. I'm at dentist.  

This #job might be a great fit for you: HR Associate Manager - Thailand -. I'm at The October in Khlong San, Bangkok. I'm at Steak Lung Nhuad in Thon Buri, Bangkok. I'm at Juicy Fruit in Bang Yi Ruea, Bangkok. I'm at Cleo's Hut @ Hive Taksin. I'm at Chill Out : Ongsa =^o^= in Bangkok. I'm at BTS Pho Nimit - @bts_skytrain in Thon Buri, Bangkok. I'm at BTS Pho Nimit - @bts_skytra

## Doc2Vec

In [233]:
train_corpus = list(read_corpus(path_play + "/bkk_demo_train"))
#test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [234]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100, dm=0)

In [235]:
model.build_vocab(train_corpus)

In [236]:
model.wv.vocab["สยาม"].count

17

In [237]:
model.wv.vocab

{'at': <gensim.models.keyedvectors.Vocab at 0x1edadbb3048>,
 'anantara': <gensim.models.keyedvectors.Vocab at 0x1edadbb3080>,
 'boat': <gensim.models.keyedvectors.Vocab at 0x1edadbb3780>,
 'in': <gensim.models.keyedvectors.Vocab at 0x1edadbb3390>,
 'bangkok': <gensim.models.keyedvectors.Vocab at 0x1edb0549630>,
 'skyline': <gensim.models.keyedvectors.Vocab at 0x1edb0549668>,
 'intersection': <gensim.models.keyedvectors.Vocab at 0x1edb05495c0>,
 'thon': <gensim.models.keyedvectors.Vocab at 0x1edb0549588>,
 'buri': <gensim.models.keyedvectors.Vocab at 0x1edb05495f8>,
 'riverside': <gensim.models.keyedvectors.Vocab at 0x1edacaac208>,
 'plaza': <gensim.models.keyedvectors.Vocab at 0x1edacaac0f0>,
 'just': <gensim.models.keyedvectors.Vocab at 0x1edacaac358>,
 'posted': <gensim.models.keyedvectors.Vocab at 0x1edacaac390>,
 'photo': <gensim.models.keyedvectors.Vocab at 0x1edacaac3c8>,
 'hospital': <gensim.models.keyedvectors.Vocab at 0x1edacaac400>,
 'the': <gensim.models.keyedvectors.Vocab a

In [238]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 1.44 s


In [239]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([-0.02327986, -0.11840101,  0.08303994, -0.06228102, -0.10223421,
       -0.09815366,  0.10287776, -0.16533646,  0.07394683,  0.09989592,
        0.00195579, -0.00995601, -0.0801132 ,  0.11721039, -0.09396103,
        0.01917049, -0.03513331, -0.03215053, -0.11901906,  0.02258802,
        0.23921788,  0.10961678,  0.10666274, -0.00177497,  0.02486449,
       -0.12101698, -0.14424342, -0.09078947, -0.16167653,  0.04435857,
       -0.00541268, -0.05320853,  0.06579401, -0.12678598, -0.02892133,
        0.12175304, -0.06346388,  0.01868443, -0.06719958,  0.13739645,
        0.02025124,  0.02039012, -0.10627395,  0.03092965, -0.11705581,
        0.05322952, -0.12682505,  0.04179654, -0.00364441,  0.17082712,
       -0.1889481 , -0.01776363,  0.1305533 ,  0.08688699,  0.08969188,
        0.03756095,  0.16234471,  0.07817725, -0.00061574, -0.08802889,
       -0.08488017, -0.08391363, -0.03528588, -0.09293895,  0.23861913,
        0.02737553,  0.08575933,  0.05112139,  0.0765162 , -0.02

In [240]:
len(train_corpus)

100

In [241]:
ranks = []
second_ranks = []
all_sims = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    all_sims.append(sims)
    
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [242]:
print(sims)

[(99, 0.9789587259292603), (81, 0.7803934812545776), (80, 0.7767412662506104), (30, 0.7755513787269592), (39, 0.7507176995277405), (68, 0.6933637857437134), (66, 0.6411511898040771), (64, 0.6374133229255676), (61, 0.6205658912658691), (53, 0.6145558953285217), (12, 0.6145064830780029), (11, 0.5940486788749695), (52, 0.5926021337509155), (77, 0.5905028581619263), (63, 0.5873938202857971), (13, 0.5717247724533081), (96, 0.5661581754684448), (10, 0.5642262101173401), (54, 0.5623686909675598), (67, 0.5583304166793823), (15, 0.5485820174217224), (40, 0.5462414622306824), (2, 0.5440451502799988), (60, 0.5241349339485168), (31, 0.5218163728713989), (41, 0.5200716257095337), (57, 0.51530522108078), (74, 0.5121198892593384), (14, 0.5116392374038696), (22, 0.4962104558944702), (37, 0.48569798469543457), (72, 0.484432190656662), (90, 0.48403453826904297), (1, 0.47834712266921997), (27, 0.470412939786911), (38, 0.4678080976009369), (8, 0.4479192793369293), (4, 0.44436952471733093), (43, 0.43712943

In [243]:
print(ranks,"\n")
print(second_ranks, "\n")
print(ranks.count(0))

[83, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 94, 0, 0, 0, 0, 0, 3, 0, 11, 49, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 79, 0, 0, 0, 0, 0, 0, 0, 0, 56, 0, 0, 0, 0, 0, 0, 0, 0] 

[(38, 0.23735788464546204), (2, 0.834402322769165), (1, 0.8101112842559814), (32, 0.6688748002052307), (2, 0.8300477266311646), (27, 0.6995557546615601), (16, 0.5887011885643005), (90, 0.7176396250724792), (30, 0.6432021856307983), (4, 0.7344663143157959), (40, 0.8675673007965088), (10, 0.8456400036811829), (4, 0.7517693042755127), (39, 0.7693517208099365), (72, 0.7327886819839478), (80, 0.7221014499664307), (27, 0.7222678661346436), (32, 0.6804893612861633), (97, 0.6539158821105957), (10, 0.7477166652679443), (38, 0.23735788464546204), (10, 0.8614174127578735), (10, 0.8530761003494263), (10, 0.6619930267333984), (72, 0.5802489519119263), (26, 0.6737961173057556), (36, 0.8181613683

In [244]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (99): «virginstar listen to the song and tell feeling at lot like love cafe dpcynl cattshirt cattshirt catradio chayanit patchayanit just began running workout using endomondo peptalk me now virginhitz and listen to the song feeling virginhitz feel free to listen and tell feeling at zap sading in huaykwang bangkok just began running workout using endomondo peptalk me now just began running workout using endomondo peptalk me now just finished running km in with endomondo endorphins»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d80,n5,mc2,s0.001,t3):

MOST (99, 0.9789587259292603): «virginstar listen to the song and tell feeling at lot like love cafe dpcynl cattshirt cattshirt catradio chayanit patchayanit just began running workout using endomondo peptalk me now virginhitz and listen to the song feeling virginhitz feel free to listen and tell feeling at zap sading in huaykwang bangkok just began running workout using endomondo peptalk me now just began running workout using 

## merge area

In [245]:
len(all_sims)

100

In [246]:
print(in_tuple(all_sims[1],50)[0])
print(in_tuple(all_sims[50],1))

0.5957437753677368
[0.688806414604187]


In [247]:
print(all_sims[0])
print(all_sims[99])

[(47, 0.24192368984222412), (38, 0.23735788464546204), (28, 0.22578632831573486), (86, 0.1855728030204773), (29, 0.17432987689971924), (55, 0.15763749182224274), (45, 0.15571080148220062), (25, 0.15338316559791565), (4, 0.15205219388008118), (79, 0.14604486525058746), (37, 0.13266216218471527), (70, 0.1196906715631485), (17, 0.11615826189517975), (26, 0.11612044274806976), (42, 0.11410404741764069), (98, 0.11325519531965256), (9, 0.10842886567115784), (32, 0.10489500313997269), (67, 0.10374873876571655), (56, 0.1029038280248642), (69, 0.10212773084640503), (73, 0.0983426570892334), (16, 0.09160755574703217), (7, 0.08722493797540665), (78, 0.08719489723443985), (1, 0.08591610193252563), (59, 0.0856863260269165), (49, 0.08286963403224945), (24, 0.0820772796869278), (40, 0.0726480484008789), (97, 0.07193568348884583), (27, 0.07075834274291992), (8, 0.06957730650901794), (72, 0.06928278505802155), (6, 0.0667196661233902), (53, 0.0639498233795166), (75, 0.05919305980205536), (89, 0.05855485

In [248]:
threshold = 0.6

In [249]:
for i in range(len(idx_color)):
    w = i-1 if (i-1)%10 != 9 else -1
    e = i+1 if ((i+1)%10 != 0) and (i+1 < len(idx_color)) else -1
    n = i-10 
    s = i+10 if i+10 < len(idx_color) else -1
    if w>0:
        similarw = (in_tuple(all_sims[i], w)[0] + in_tuple(all_sims[w], i)[0])/2
        if similarw > threshold:
            idx_color[str(w)][4] = idx_color[str(i)][4]
    if e>0:
        similare = (in_tuple(all_sims[i], e)[0] + in_tuple(all_sims[e], i)[0])/2
        if similare > threshold:
            idx_color[str(e)][4] = idx_color[str(i)][4]
    if n>0:
        similarn = (in_tuple(all_sims[i], n)[0] + in_tuple(all_sims[n], i)[0])/2
        if similarn > threshold:
            idx_color[str(n)][4] = idx_color[str(i)][4]
    if s>0:
        similars = (in_tuple(all_sims[i], s)[0] + in_tuple(all_sims[s], i)[0])/2
        if similars > threshold:
            idx_color[str(s)][4] = idx_color[str(i)][4]

In [250]:
color_idx

{}

In [252]:
for i in range(10):
    s = ""
    for j in range(10):
        if int(idx_color[str((i*10)+j)][4]) < 10:
            s += " "
        s += idx_color[str((i*10)+j)][4] +" "
    print(s)

 0 10  1  3  3  5  6  7  8  9 
10 10  1 13  3 15 16 17 18 19 
20 10 10 23 24 25 25 27 27 28 
30 30 32 33 34 35 25 27 27 27 
40 40 42 43 44 45 46 47 48 49 
40 51 52 52 54 55 56 57 58 49 
40 40 62 63 64 65 66 67 67 49 
70 71 72 72 64 75 76 77 78 79 
80 80 82 83 84 85 86 87 88 89 
90 91 92 93 94 95 96 97 98 99 


In [216]:
for i in range(10):
    s = ""
    for j in range(10):
        if int(idx_color[str((i*10)+j)][4]) < 10:
            s += " "
        s += idx_color[str((i*10)+j)][4] +" "
    print(s)

 0  1  2  3  3  3  3  6  8  8 
10  1 12 13 14  3  3 17  8  8 
20 30 21 23 23  3  3  3  8 29 
30 30 30 33 33  3  3  3  3  3 
40 30 30 43 44 45 46 47 47  3 
50 30 30 30 54 55 56 47 58  3 
60 30 62 30 64 65 66 66 66 69 
70 71 72 73 74 75 66 77 78 79 
80 80 82 83 84 85 86 96 88 89 
80 91 92 93 94 94 96 96 98 89 


In [254]:
idx_color["26"]

[(13.745779999999998, 100.50460500000001),
 (13.745779999999998, 100.51460500000002),
 (13.755779999999998, 100.50460500000001),
 (13.755779999999998, 100.51460500000002),
 '25']

## Word2Vec

In [8]:
from __future__ import print_function
from gensim.models import KeyedVectors

In [ ]:
# Creating the model
en_model = KeyedVectors.load_word2vec_format('glove.twitter.27B.50d.txt')

# Getting the tokens 
en_words = []
for word in th_model.vocab:
    th_words.append(word)

In [ ]:
# Printing out number of tokens available
print("Number of Tokens: {}".format(len(th_words)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(th_model[th_words[0]])
))

# Print out the vector of a word 
print("Vector components of a word: {}".format(
    th_model[th_words[0]]
))

In [ ]:
#Pick a word 
find_similar_to = 'แพง'

# Finding out similar words [default= top 10]
for similar_word in th_model.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

In [ ]:
# Test words 
word_add = ['กรุงเทพ', 'ไทย']
word_sub = ['จำกัด']

# Word vector addition and subtraction 
for resultant_word in th_model.most_similar(
    positive=word_add, negative=word_sub
):
    print("Word : {0} , Similarity: {1:.2f}".format(
        resultant_word[0], resultant_word[1]
    ))

In [ ]:
prew = []
prew.append(np.zeros(300))
for i in range(1,len(dictionary)):
    if(reverse_dictionary[i] in th_model.vocab):
        prew.append(th_model[reverse_dictionary[i]])
    else:
        prew.append(np.zeros(300))
print(len(prew))

In [ ]:
ft_pre = prew.copy()
for i in range(len(th_model.vocab)):
    if(th_words[i] not in dictionary.keys()):
        ft_pre.append(th_model[th_words[i]])
    else:
        ft_pre.append(np.zeros(300))
print(len(ft_pre))

In [ ]:
x=sequence.pad_sequences(dataset, maxlen=1000, dtype='int32', padding='post', truncating='pre', value=0.)
y=np.array(pd.get_dummies(df[1]))